### Hipparcos HR - diagram

In this notebook, we will create Python codes that include various operations performed on the Hipparcos data. These functions can be used from other notebooks. 

### 1. Importing and preparing data 

This function imports selected columns from the Hipparcos catalog into a DataFrame and prepares the columns by setting the data type of the numerical columns.  

In [1]:
def Import_Hipparcos(filename):

    #defining column names
    new_column_names = ['Hip_No', 'Alpha', 'Delta','Vmag','Plx', 'e_Plx', 'B-V','e_B-V', 'ccdm_h','Spectral_type']

    #importing data
    hip_all_stars = pd.read_csv(file, header = None, sep =',',
                    usecols = [1,2,3,4,5,6,7,8,9,10],  
                    names = new_column_names,
                    low_memory = False)

    #changing column types
    col_list = ['Vmag', 'Plx', 'e_Plx', 'B-V', 'e_B-V']
    for  col in col_list:
        hip_all_stars[col] = pd.to_numeric(hip_all_stars[col],  errors = 'coerce') 

    #selecting only single stars with no ccdm_h flag
    df = hip_all_stars.loc[hip_all_stars['ccdm_h']==' ']

    #dropping ccdm_h column
    hip = df.drop(['ccdm_h'], axis = 1) 
    return(hip)    

### 2.  Calculating the absolute stellar magnitudes

This function calculates a new column in the DataFrame containing the absolute stellar magnitudes (Mv) of the Hipparcos stars. 

In [2]:
def Find_AbsMag(df):
    #selecting stars with positive Plx and with the relative errors in Plx < 0.80
    hip_stars = df.loc[(df['Plx']>0) & (df['e_Plx']/df['Plx'].abs()<0.80)].copy()

    #creating a new column with the absolute magnitude
    hip_stars['Mv'] = hip_stars['Vmag'] + 5 - 5*np.log10(1000/hip_stars['Plx'])
    
    #rounding numbers
    hip_stars = hip_stars.round({'Mv': 2})
    return(hip_stars)

### 3. The HR - diagram 

This function plots the Hertzsprung-Russell (HR) diagram presenting the change in the stellar luminosity, absolute magnitude (Mv), with the spectral type expressed through the B-V color index. 

In [3]:
def HR_plot(df):

    #selecting stars with the relative errors in parallax: e_Plx/abs(Plx) < 10 %
    hip1 = df.loc[(df['e_Plx']/df['Plx'].abs())<0.10]
    hip_plot = hip1.loc[hip1['B-V']< 2]
    #marking selected stars
    sns.set(rc={'figure.figsize':(10,8)})
    sns.scatterplot(x ='B-V', y ='Mv', data = hip_plot, hue ='B-V', size ='B-V', sizes = (5,5), palette='turbo')

    plt.ylim(15,-8)
    plt.xlim(-0.5,2.3)
    plt.title('Hertzsprung-Russell Diagram')
    plt.annotate('Main sequence', xy=(0.4,7.0), rotation = -35)
    plt.annotate('Giants', xy=(1.0,-0.2))
    plt.annotate('Horizontal branch ', xy=(0.8,4.0))
    plt.annotate('White dwarfs', xy=(-0.2,11.0))
    plt.annotate('Red dwarfs', xy=(1.6,9.0))
    plt.annotate('Blue supergiants', xy=(-0.35,-1.5))
    #Sun's position at 0.63, 4.8
    yellow_dot = plt.plot(0.63,4.8, marker='o', color='yellow', linestyle='none')
    plt.legend(yellow_dot, ['Sun'])